In [ ]:
# Import necessary libraries
# Let's try and remove all the tkinter and make this barebonse
import arcpy
import os
import shutil
import re
# import tkinter as tk
# from tkinter import ttk
# from tkinter import Label, Button, Entry, filedialog, messagebox

import time
# from PIL import ImageTk, Image

In [ ]:
# Set Input Variables
fiber = r"\\nadnp1b_cifs.corp.intranet\NNSGIS3\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.fibercable"
route = r"\\nadnp1b_cifs.corp.intranet\NNSGIS3\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.Route"
mh = r"\\nadnp1b_cifs.corp.intranet\NNSGIS3\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.MH_HH"
site = r"\\nadnp1b_cifs.corp.intranet\NNSGIS3\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.Building"


In [ ]:
# Define the function to simplify the customer string
def simplify_customer(customer):
    """
    Simplifies a customer name by removing punctuation and legal suffixes.

    Args:
        customer (str): The name of the customer to simplify.

    Returns:
        str: The simplified customer name.
    """
    # Remove any punctuation and extra whitespace
    customer = re.sub(r'[^\w\s]', '', customer).strip()
    # Split the string into a list of words
    words = customer.split()
    # Remove any common legal suffixes
    suffixes = ['inc', 'llc', 'ltd', 'corp', 'limited']
    words = [word.upper() if word.lower() not in suffixes else '' for word in words]
    # Rejoin the words into a single string without spaces
    simplified_customer = ''.join(words)
    return simplified_customer

In [ ]:
class metadata_create:
    def __init__(self, path, tags, summary, description, accessConstraints):
        self.path = path
        self.metadata = arcpy.metadata.Metadata(self.path)
        self.metadata.tags = tags
        self.metadata.summary = summary
        self.metadata.description = description
        self.metadata.accessConstraints = accessConstraints
        self.metadata.save()

In [ ]:
# Define the function to create the deliverable files
def create_deliverable():
    # Create the progress bar
    # progress_bar = ttk.Progressbar(root, orient="horizontal", length=100, mode="determinate")
    # progress_bar.grid(row=11, column=0, columnspan=3)

    # Get the values from the GUI elements
    customer = input("Enter the customer name: ")
    simplified_customer = simplify_customer(customer)  # simplify the customer string
    tasknum = input("Enter the task number: ")
    if not tasknum:
        messagebox.showinfo("Error", "Task number cannot be empty.")
        return
    if tasknum[0].isdigit():
        messagebox.showinfo("Error", "Task number cannot start with a number.")
        return
    orderid = input("Enter the order ID: ")
    tag_span = input("Enter the tag span: ")
    PROCircuitName = input("Enter the PRO Circuit Name: ")
    fibercount_str = input("Enter the fiber count: ")
    try:
        fibercount = int(fibercount_str)
    except ValueError:
        messagebox.showinfo("Error", "Fiber count must be a number.")
        return
    fibertype = input("Enter the fiber type: ")
    fibers = input("Enter the assigned fibers: ")
    azcities = input("Enter the AZ cities: ")
    # Need to add error handling for tag_lh
    tag_lh = input("Idntify the Longhaul Route Name: ")
    tag_lh = tag_lh.replace("-", "_")
    requestor = input("Who is the requestor: ")
    parent_dir = input("Define the path to the parent directory: ")



In [ ]:
# Define the metadata strings
tag_info = "LUMEN LH ROUTE MH SPLICE SITES"
span = "SPAN"
timestamp = time.strftime("%m-%d-%Y")
# Modified Directory Name
directory_name = f"{tasknum}_{PROCircuitName}_LUMEN_{tag_lh}_{simplified_customer}_{span}_{tag_span}"

# Define the metadata file path
# construct the full path to the working directory
path = os.path.join(parent_dir, directory_name)

if os.path.exists(path):
    # directory already exists, delete it
    shutil.rmtree(path)
    arcpy.AddMessage("Directory overwritten.")

# create the new directory
os.mkdir(path)
arcpy.AddMessage("Directory created.")
# worgdb = arcpy.management.CreateFileGDB(path, f"{directory_name}", "CURRENT")

In [ ]:
# Define GDB name and Create the file geodatabase
worgdb = arcpy.management.CreateFileGDB(path, f"LUMEN_{tag_lh}_{simplified_customer}_{span}_{tag_span}", "CURRENT")
file_path = os.path.join(path, "metadata.txt")

# # Update progress bar value
# progress_bar["value"] = 20
# progress_bar.update()

# Write the metadata to the file
tag = f"{tag_info} {tag_lh} {span} {tag_span} {PROCircuitName}"
summary = f"{span} {tag_span} Overpull from {azcities} for {customer} deliverable {orderid}: on fibers {fibers}."
description = f"{tasknum} {requestor} {timestamp}"
accessConstraints = f"Disclose and distribute only to LUMEN and {customer} employees and authorized persons working on behalf of LUMEN and {customer} having a legitimate business need to know. Disclosure and distribution is prohibited without authorization."
with open(file_path, "w") as file:
    file.write("\n".join([tag, summary, description, accessConstraints]))

# Create Medadata
with open(file_path, 'r') as f:
    metadata = f.readlines()

tags = metadata[0].strip()
summary = metadata[1].strip()
description = metadata[2].strip()
accessConstraints = metadata[3].strip()

In [ ]:
# Export the FiberCable based no the PRO Circuit Name

pcresults = ""
fieldMappings = ""
pcresults = arcpy.FeatureClassToFeatureClass_conversion(fiber, worgdb, f"{PROCircuitName}",
                                                        "SEGMENTID IN (SELECT SEGMENTIDFKEY FROM NX_DATA.FIBER WHERE NX_DATA.FIBER.SIGNALIPID IN(SELECT IPID FROM NX_DATA.SIGNAL WHERE NX_DATA.SIGNAL.CIRCUITNAME LIKE '%" + PROCircuitName + "%'))",
                                                        fieldMappings)


In [ ]:
# Export the MH based on the FiberCable From & To structures
idCur = arcpy.SearchCursor(pcresults)
cnt = 0
results = []
for idResult in idCur:
    results.append(idResult.FROM_STRUCTURE)
    results.append(idResult.TO_STRUCTURE)
    cnt = cnt + 1
del idCur
pointquery = ("CLLI IN " + str(results).replace("[", "(").replace("]", ")"))

mhsplice_del = arcpy.conversion.FeatureClassToFeatureClass(mh, worgdb,
                                                            fr"LUMEN_MH_SPLICE_{tag_lh}_{span}_{tag_span}",
                                                            pointquery,
                                                            'CLLI "CLLI-CAPID" true true false 100 Text 0 0,First,#,MH_HH_Layer2,CLLI,0,100;LONGITUDE "Longitude" true true false 8 Double 8 38,First,#,MH_HH_Layer2,LONGITUDE,-1,-1;LATITUDE "Latitude" true true false 8 Double 8 38,First,#,MH_HH_Layer2,LATITUDE,-1,-1;SPAN_ID "SPAN_ID" true true false 5 Text 0 0,First,#',
                                                            '')
# Calculate the Span ID for the MH Splice
arcpy.management.CalculateField(mhsplice_del, "SPAN_ID", fr'"{tag_span}"', "PYTHON3", '', "TEXT",
                                "NO_ENFORCE_DOMAINS")

# Add Metadata to the MH Splice Feature Class
deliverable = metadata_create(mhsplice_del, tags, summary, description, accessConstraints)

In [ ]:
# Export the Sites based on the FiberCable From & To structures
site_del = arcpy.conversion.FeatureClassToFeatureClass(site, worgdb, fr"LUMEN_SITES_{tag_lh}_{span}_{tag_span}",
                                                        pointquery,
                                                        r'CLLI "CLLI" true true false 100 Text 0 0,First,#,J:\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.building,CLLI,0,100;ADDRESS "Address" true true false 255 Text 0 0,First,#,J:\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.building,ADDRESS,0,255;CITY "City" true true false 150 Text 0 0,First,#,J:\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.building,CITY,0,150;TYPE "TYPE" true true false 20 Text 0 0,First,#,J:\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.building,BUILDING_TYPE,0,20;LONG "LONG" true true false 8 Double 8 38,First,#,J:\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.building,LONGITUDE,-1,-1;LAT "LAT" true true false 8 Double 8 38,First,#,J:\ArcMap\Connections\NXPRD3_GISREAD.sde\NX_DATA.TelecomDatasetProjection\NX_DATA.building,LATITUDE,-1,-1',
                                                        '')

# Calculate the Span ID for the Sites
arcpy.management.CalculateField(site_del, "SPAN_ID", fr'"{tag_span}"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

# Add Metadata to the Sites Feature Class
deliverable = metadata_create(site_del, tags, summary, description, accessConstraints)

In [ ]:
# Get the Buidign Count
bldg_count = arcpy.management.GetCount(in_rows=site_del)

# open the file with append mode
with open(file_path, "a") as f:
    f.write("\nBuilding Count: " + str(bldg_count) + "\n")
    f.close()

In [ ]:
# Export the Route based on Share A Line Segment With the FiberCable

route_sel = arcpy.management.SelectLayerByLocation(route, "SHARE_A_LINE_SEGMENT_WITH", pcresults, None,
                                                    "NEW_SELECTION", "NOT_INVERT")
arcpy.SetParameter(2, pcresults)

# Feature Class to Feature Class
route_del = arcpy.conversion.FeatureClassToFeatureClass(route_sel, worgdb,
                                                        fr"LUMEN_LH_ROUTE_{tag_lh}_{span}_{tag_span}", '',
                                                        'INSTALL "INSTALL" true false false 255 Text 0 0,First,#,Route,INSTALLATION_METHOD,0,255;INTERCITY "INTERCITY" true true false 100 Text 0 0,First,#,Route,INTERCITY_MARKET,0,100;LEGACY "LEGACY" true false false 255 Text 0 0,First,#,Route,NETWORK_BUILDER,0,255;SPAN_NAME "SPAN NAME" true true false 255 Text 0 0,First,#,Route,SPAN_NAME,0,255;ORDER_ID "ORDER_ID" true true false 255 Text 0 0,First,#;CIRCUIT_ID "CIRCUIT_ID" true true false 255 Text 0 0,First,#;FIBERCOUNT "FIBERCOUNT" true true false 255 Long 0 0,First,#;FIBER_TYPE "FIBER_TYPE" true true false 255 Text 0 0,First,#;FIBERS "FIBERS" true true false 255 Text 0 0,First,#;A_Z_ID "A_Z_ID" true true false 255 Text 0 0,First,#',
                                                        '')
# Add Metadata to the Route Feature Class
deliverable = metadata_create(route_del, tags, summary, description, accessConstraints)

# Calculate the fields for the Route Feature Class
arcpy.management.CalculateField(route_del, "A_Z_ID", fr'"{azcities}"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(route_del, "ORDER_ID", fr'"{orderid}"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(route_del, "CIRCUIT_ID", fr'"{PROCircuitName}"', "PYTHON3", '', "TEXT",
                                "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(route_del, "FIBERS", fr'"{fibers}"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(route_del, "FIBERCOUNT", fr'"{fibercount}"', "PYTHON3", '', "LONG",
                                "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(route_del, "FIBER_TYPE", fr'"{fibertype}"', "PYTHON3", '', "TEXT",
                                "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(route_del, "SPAN_ID", fr'"{tag_span}"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")



In [ ]:
print("Export Complete", f"Your data has been exported to {worgdb}")